# To Be or Not To Be

In [155]:
import numpy as np

In [156]:
import pandas as pd

In [157]:
import matplotlib.pyplot as plt

In [158]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [159]:
dfShake=pd.read_csv("file://localhost/Users/anissakhan/Dropbox/Nissa_stuff/KU_CS/EECS731/731-Project-2-Khan/Data/RawData/Shakespeare_data.csv")

In [160]:
dfShake.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


## Data Cleaning

If Player doesn't exist (Player=NaN), delete those observations, because the associated PlayerLine is not spoken by anyone (example: PlayerLine=ACT I).

If ActSceneLine doesn't exist (ActSceneLine=NaN), delete those observations, because the associated PlayerLine is a stage direction and therefore not spoken by anyone (example: ActSceneLine=SCENE III. London. The palace.).

In the dropna command, inplace=True because we want the dataframe itself to be changed (as opposed to a copy of the dataframe)

In [161]:
dfShake.dropna(subset = ["Player"], inplace=True)

In [162]:
dfShake.dropna(subset = ["ActSceneLine"], inplace=True)

Verify there are no more missing values in the dataset. Since the number of entries matches the non-null count for all variables, we know there are no more missing values.

In [163]:
dfShake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105152 entries, 3 to 111394
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Dataline          105152 non-null  int64  
 1   Play              105152 non-null  object 
 2   PlayerLinenumber  105152 non-null  float64
 3   ActSceneLine      105152 non-null  object 
 4   Player            105152 non-null  object 
 5   PlayerLine        105152 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 5.6+ MB


In [164]:
dfShake.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil


In [165]:
dfShake=dfShake.reset_index()

In [166]:
dfShake

,index,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
1,4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
2,5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
3,6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
4,7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil
...,...,...,...,...,...,...,...
105147,111390,111391,A Winters Tale,38.0,5.3.179,LEONTES,"Is troth-plight to your daughter. Good Paulina,"
105148,111391,111392,A Winters Tale,38.0,5.3.180,LEONTES,"Lead us from hence, where we may leisurely"
105149,111392,111393,A Winters Tale,38.0,5.3.181,LEONTES,Each one demand an answer to his part
105150,111393,111394,A Winters Tale,38.0,5.3.182,LEONTES,Perform'd in this wide gap of time since first


Convert Player and PlayerLine to string

In [167]:
dfShake.dtypes

index                 int64
Dataline              int64
Play                 object
PlayerLinenumber    float64
ActSceneLine         object
Player               object
PlayerLine           object
dtype: object

In [168]:
dfShake['Player'] = dfShake['Player'].astype("string")
dfShake['PlayerLine'] = dfShake['PlayerLine'].astype("string")

In [169]:
dfShake.dtypes

index                 int64
Dataline              int64
Play                 object
PlayerLinenumber    float64
ActSceneLine         object
Player               string
PlayerLine           string
dtype: object

There is a problem with the data: The player name "ANTIPHOLUS OF EPHESUS" has been inadvertently split. "ANTIPHOLUS" is split off from the rest of the name and placed incorrectly in PlayerLine

In [170]:
dfShake.PlayerLine = dfShake.PlayerLine.str.strip()

In [171]:
count=0
for i in range(0,(len(dfShake['PlayerLine']))):
    if(dfShake['PlayerLine'][i] == 'ANTIPHOLUS'):
        count=count+1
        if count<3:
            print(dfShake['PlayerLine'][i], dfShake['Dataline'][i])
            
print('Times this error occurs =', count) 

ANTIPHOLUS 22600
ANTIPHOLUS 22612
Times this error occurs = 175


In [172]:
for i in range(0,(len(dfShake['PlayerLine']))):
    if(dfShake['PlayerLine'][i] == 'ANTIPHOLUS'):
        dfShake.drop([i],inplace=True, axis=0)

In [173]:
dfShake=dfShake.reset_index()

In [174]:
for i in range(0,len(dfShake['Player'])):
    if(dfShake['Player'][i] == 'OF EPHESUS'):
        dfShake['Player'][i]='ANTIPHOLUS OF EPHESUS'

There is also a problem with the player "DROMIO OF SYRACUSE". His whole name isn't stored in Player consistently

In [175]:
count=0
for i in range(0,(len(dfShake['Player']))):
    if(dfShake['Player'][i] == 'OF SYRACUSE'):
        count=count+1
        if count<3:
            print(dfShake['Player'][i], dfShake['Dataline'][i])

print('Times this error occurs =', count) 

OF SYRACUSE 22601
OF SYRACUSE 22602
Times this error occurs = 269


In [176]:
for i in range(0,len(dfShake['Player'])):
    if(dfShake['Player'][i] == 'OF SYRACUSE'):
        dfShake['Player'][i]='DROMIO OF SYRACUSE'

In [177]:
#dfShake.to_csv('test.csv')

Combine player lines so that the entire line is contained in a single observation

In [178]:
dfShake=dfShake.groupby(['Play','PlayerLinenumber', 'Player'], as_index = False).agg({'PlayerLine': ' '.join})

In [179]:
dfShake

,Play,PlayerLinenumber,Player,PlayerLine
0,A Comedy of Errors,1.0,ADRIANA,"Neither my husband nor the slave return'd, Tha..."
1,A Comedy of Errors,1.0,AEGEON,"Proceed, Solinus, to procure my fall And by th..."
2,A Comedy of Errors,1.0,ANGELO,"I am sorry, sir, that I have hinder'd you, But..."
3,A Comedy of Errors,1.0,ANTIPHOLUS OF EPHESUS,"Good Signior Angelo, you must excuse us all, M..."
4,A Comedy of Errors,1.0,DROMIO OF SYRACUSE,The gold I gave to Dromio is laid up Safe at t...
...,...,...,...,...
22212,macbeth,61.0,MALCOLM,Dispute it like a man.
22213,macbeth,62.0,MACDUFF,"I shall do so, But I must also feel it as a ma..."
22214,macbeth,63.0,MALCOLM,Be this the whetstone of your sword: let grief...
22215,macbeth,64.0,MACDUFF,"O, I could play the woman with mine eyes And b..."


## Feature Engineering

Create new variable that contains the number of words in each player line. This can be useful if there are certain characters that tend to talk a lot more than others.

In [180]:
quoteLength = []
for x in range(0,len(dfShake['PlayerLine'])):
    quoteLength.append(len(dfShake['PlayerLine'][x].split()))
    #dfShake.assign(quoteLength=len(dfShake['PlayerLine'][x].split()))
dfShake['quoteLength'] = quoteLength

In [181]:
dfShake.head()

,Play,PlayerLinenumber,Player,PlayerLine,quoteLength
0,A Comedy of Errors,1.0,ADRIANA,"Neither my husband nor the slave return'd, Tha...",71
1,A Comedy of Errors,1.0,AEGEON,"Proceed, Solinus, to procure my fall And by th...",16
2,A Comedy of Errors,1.0,ANGELO,"I am sorry, sir, that I have hinder'd you, But...",25
3,A Comedy of Errors,1.0,ANTIPHOLUS OF EPHESUS,"Good Signior Angelo, you must excuse us all, M...",166
4,A Comedy of Errors,1.0,DROMIO OF SYRACUSE,The gold I gave to Dromio is laid up Safe at t...,186


Summary statistics for the quote length variable:

In [182]:
dfShake['quoteLength'].describe()

count    22217.000000
mean        35.496737
std         53.561607
min          1.000000
25%          8.000000
50%         17.000000
75%         41.000000
max       1555.000000
Name: quoteLength, dtype: float64

Check how many times there are 1 word lines:

In [183]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['quoteLength'][i] == 1):
        #print(dfShake.iloc[i])
        count=count+1

print('Count of 1 word lines =', count) 

Count of 1 word lines = 215


Check how many times there are 1555 word lines:

In [184]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['quoteLength'][i] == 1555):
        print(dfShake.iloc[i])
        count=count+1

print('Count of 1555 word lines =', count) 

Play                                                          Henry V
PlayerLinenumber                                                    1
Player                                                         Chorus
PlayerLine          O for a Muse of fire, that would ascend The br...
quoteLength                                                      1555
Name: 6172, dtype: object
Count of 1555 word lines = 1


Check how many times there are < and >= an average # word lines (35):

In [185]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['quoteLength'][i] < 35):
        #print(dfShake.iloc[i])
        count=count+1

print('Count of <avg lines =', count) 

Count of <avg lines = 15701


In [186]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['quoteLength'][i] >= 35):
        #print(dfShake.iloc[i])
        count=count+1

print('Count of >=avg lines =', count) 

Count of >=avg lines = 6516


In [187]:
dfShake.head()

,Play,PlayerLinenumber,Player,PlayerLine,quoteLength
0,A Comedy of Errors,1.0,ADRIANA,"Neither my husband nor the slave return'd, Tha...",71
1,A Comedy of Errors,1.0,AEGEON,"Proceed, Solinus, to procure my fall And by th...",16
2,A Comedy of Errors,1.0,ANGELO,"I am sorry, sir, that I have hinder'd you, But...",25
3,A Comedy of Errors,1.0,ANTIPHOLUS OF EPHESUS,"Good Signior Angelo, you must excuse us all, M...",166
4,A Comedy of Errors,1.0,DROMIO OF SYRACUSE,The gold I gave to Dromio is laid up Safe at t...,186


Create 2 new features: a variable that contains the average sentence length of each quote (playerLine), and a variable that contains the number of sentences of each quote (playerLine).

In [188]:
def Avg(lst): 
    return sum(lst) / len(lst)

AvgSentLen=[] #New feature: avg sentence length
NumSentPerLine=[] #New feature: number of sentences per quote
sentenceLen=[] #sentence Lengths per quote
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    tempQuote=dfShake['PlayerLine'][i].split()
    
    for j in range(0,len(tempQuote)):
        count=count+1
        if("." in tempQuote[j] or "?" in tempQuote[j] or "!" in tempQuote[j] or "--" in tempQuote[j]):
            sentenceLen.append(count)
            count=0
    if(len(sentenceLen)>0):
        AvgSentLen.append(Avg(sentenceLen))
        NumSentPerLine.append(len(sentenceLen))
    else:
        AvgSentLen.append(0)
        NumSentPerLine.append(0)
    sentenceLen.clear()

    
dfShake['AvgSentLen'] = AvgSentLen
dfShake['NumSentPerLine']=NumSentPerLine

In [189]:
dfShake.head()

,Play,PlayerLinenumber,Player,PlayerLine,quoteLength,AvgSentLen,NumSentPerLine
0,A Comedy of Errors,1.0,ADRIANA,"Neither my husband nor the slave return'd, Tha...",71,10.142857,7
1,A Comedy of Errors,1.0,AEGEON,"Proceed, Solinus, to procure my fall And by th...",16,16.000000,1
2,A Comedy of Errors,1.0,ANGELO,"I am sorry, sir, that I have hinder'd you, But...",25,25.000000,1
3,A Comedy of Errors,1.0,ANTIPHOLUS OF EPHESUS,"Good Signior Angelo, you must excuse us all, M...",166,20.750000,8
4,A Comedy of Errors,1.0,DROMIO OF SYRACUSE,The gold I gave to Dromio is laid up Safe at t...,186,13.285714,14


Check how many rows contain 0 sentence length. After visual observation of the quotes whose AvgSentLen is 0, the quotes are not empty. However, they end in a value different from ?, !, --, . so there is no good way to adjust the sentence length function without sacrificing the rest of the data. Therefore, since there are only 26 observations for which that happens, I drop them from the data.

In [190]:
count=0
for i in range(0,len(dfShake['AvgSentLen'])):
    if(dfShake['AvgSentLen'][i] < 1):
        #print(dfShake.iloc[i])
        count=count+1
        dfShake.drop([i],inplace=True, axis=0)

print('Count of 0 sentence length lines =', count) 

Count of 0 sentence length lines = 26


Create a feature that gives the frequency of the Player's lines (ie if Player has 10 lines in the play, this variable's value will be 10)

## Classification

In [191]:
feature_cols = ['quoteLength', 'AvgSentLen', 'NumSentPerLine']
X = dfShake[feature_cols] # Features
y = dfShake.Player # Target variable

In [192]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [193]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [194]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.007810153199158907
